In [ ]:
import json
import os
import requests

In [ ]:
HOST = "http://127.0.0.1"
PORT = "3012"

WEAVIATE_ENDPOINT = f"{HOST}:{PORT}/api/weaviate"
EMBED_ENDPOINT = f"{HOST}:{PORT}/api/embed"

# gemeente = "ridderkerk"
# BASE_PATH_INIT = f"/Volumes/Samsung_T5/data/{gemeente}"

# TODO!
# gemeente = "haarlem"
# BASE_PATH_INIT = f"/Volumes/Drive/data/{gemeente}"

gemeente = "buren"
BASE_PATH_INIT = f"/Users/personal/Desktop/scriptie/notebooks/final/{gemeente}"

# gemeente = "nijmegen"
# BASE_PATH_INIT = f"/Users/personal/Desktop/scriptie/notebooks/data/{gemeente}"

In [ ]:
# When creating transcript collection, uncomment the named vector config in WeaviateClient.py
def create_transcript_collection():
    requests.post(
        f"{WEAVIATE_ENDPOINT}/createCollectionTranscripts",
        json={
            "name": "TranscriptsV2",
            "vector_index_hnsw": True,
            "distance_config": "cosine",
            "properties": [
                {"name": "text", "data_type": "text"},
                {"name": "start", "data_type": "number"},
                {"name": "end", "data_type": "number"},
                {"name": "code", "data_type": "text"},
                {"name": "year", "data_type": "text"},
                {"name": "government", "data_type": "text"},
                {"name": "type", "data_type": "text"},
                {"name": "speaker", "data_type": "text"},
            ],
        },
    )


def create_speaker_collection():
    requests.post(
        f"{WEAVIATE_ENDPOINT}/createCollection",
        json={
            "name": "Speakers",
            "vector_index_hnsw": True,
            "distance_config": "cosine",
            "properties": [
                {"name": "government", "data_type": "text"},
                {"name": "name", "data_type": "text"},
            ],
        },
    )

In [ ]:
create_transcript_collection()
# create_speaker_collection()

In [ ]:
def get_embedding(text):
    body = {"text": [text]}
    r = requests.post(EMBED_ENDPOINT, json=body)
    if not r.status_code == 200:
        print("Error embedding!")
        raise Exception("Error embedding")

    return r.json()["embeddings"][0]

In [ ]:
def add_transcripts():
    for meeting_type in os.listdir(BASE_PATH_INIT):
        if meeting_type.startswith("."):
            continue
        BASE_PATH = f"{BASE_PATH_INIT}/{meeting_type}"
        for year in os.listdir(BASE_PATH):
            if year.startswith("."):
                continue
            if not os.path.isdir(f"{BASE_PATH}/{year}") or not os.path.isdir(
                f"{BASE_PATH}/{year}/finalObjects"
            ):
                continue
            print("Doing", gemeente, meeting_type, year)
            for meeting in os.listdir(f"{BASE_PATH}/{year}/finalObjects"):
                with open(f"{BASE_PATH}/{year}/finalObjects/{meeting}", "r") as fof:
                    try:
                        objects_with_embedding = json.load(fof)
                    except Exception as e:
                        print(f"Error at: {meeting}", e)
                        continue
                    requests.post(
                        f"{WEAVIATE_ENDPOINT}/insert",
                        json={
                            "collection": "TranscriptsV2",
                            "objects": objects_with_embedding,
                        },
                    )


In [ ]:
add_transcripts()